In [1]:
# system libraries
import sys
import os
import subprocess

# 3rd party libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2
from datetime import datetime
import time

# our custom libraries
sys.path.append('../python_libraries/')

from oscilloscope.oscilloscope import *
from spectrum_analyzer.spectrum_analyzer import *

from SLM.SLM import *
from HG_modes.HG_modes import *

from alignment.alignment_v2 import * 
from DMD_autoGUI.DMD_autoGUI import * 
from oscilloscope.oscilloscope import *

## our custom classes (DMD sources)
# sys.path.append('classes/')
# from classes.dataset import *

C:\ProgramData\Anaconda3\envs\lab\lib\site-packages\pyglet\media\codecs\wmf.py:838: UserWarning: [WinError -2147417850] Cannot change thread mode after it is set
  warnings.warn(str(err))


In [2]:
## to do:
# 1) threshold autoalign
# 2) set up vertical resolution scope 

**Define name of measurement, data folders, DMD sequences folder and number of LO modes**

In [3]:
name_measurement = '20x20' #'LO_basis_20_20'

save_folder = r"D:\measurements\measurements\p" + str(datetime.today().strftime('%Y%m%d')) + '\p' + str(datetime.today().strftime('%H%M%S')+ '/' + str(name_measurement)+'/') 
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

In [5]:
folder_sequences = r"data/sequences/"
sequences = np.arange(0,72,1)

In [6]:
LO_modes_m, LO_modes_n = 21, 21

**Initialize instruments libraries**

In [11]:
n_points_oscilloscope = 55000 #215000 # 475000
scope = Oscilloscope(
    n_points_oscilloscope,
    binary = True,
    channels = 'CH1,CH2,CH3,CH4'
)

Connected to: 
 TEKTRONIX,MSO5104,C010172,CF:91.1CT FV:6.8.1 Build 3

Reading channel: 
 CH1,CH2,CH3,CH4  

Data format: 
 RIB

Points per frame: 
 55000 

Info measurement: 
 2;16;BIN;RI;MSB;"Ch4, DC coupling, 400.0mV/div, 4.400ms/div, 55000 points, Sample mode";55000;Y;"s";800.0000E-9;0.0000;1090;"V";62.5000E-6;-24.7040E+3;0.0000;1
 



In [13]:
slm = SLM(screen_ID=1,
          folder_holograms= r'C:\Users\dacostafilho\Desktop\python_libraries\alignment\holograms/'
)

C:\ProgramData\Anaconda3\envs\lab\lib\site-packages\pyglet\image\codecs\wic.py:292: UserWarning: [WinError -2147417850] Cannot change thread mode after it is set
  warnings.warn(str(err))


In [14]:
program_DMD_GUI = Program_DMD_GUI(
    folder_sequences =  folder_sequences,
    sequence_temporary = "00_current_seq.txt",
    sequence_alignment_square = "00_alig_square.txt"
)

In [15]:
spectrum = SpectrumAnalyzer()
alignment = Alignment(slm,
                      spectrum,
                      log_folder = save_folder,
                      plots = False,
                      detailed_printing = False,
                      renorm = False,
                      threshold=0.013)

Connected to: 
 Rigol Technologies,DSA815,DSA8A211500380,00.01.19.00.02



**Manual setup alignment**

In [ ]:
slm.update_mode(0,0, zero_shift_angle=False)
# slm.update_mode(1,0, zero_shift_angle=True)
# slm.update_mode(0,1, zero_shift_angle=True)

# slm.update_mode(20,20, zero_shift_angle=True)

**Acquisition**

In [ ]:
# 1) NOTE: before running, open DMD GUI software and set it to pattern on the fly mode

In [16]:
print('sequences: ',sequences)
print('modes m: ',LO_modes_m)
print('modes n: ',LO_modes_n)

sequences:  [22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45
 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69
 70 71]
modes m:  21
modes n:  21


In [ ]:
# program_DMD_GUI.start_alignment_sequence()
# alignment.perform(idx = 0, save = True)

In [17]:
counter_autoalignment = 0

for seq_n in sequences:
    print('\n sequence {}/{}'.format(seq_n, len(sequences)))
    if (seq_n % 10) == 0: # perform auto-alignement 
        program_DMD_GUI.start_alignment_sequence()
        alignment.perform(idx = counter_autoalignment, save = True)
        counter_autoalignment += 1

    program_DMD_GUI.start_sequence(str(seq_n)+'.txt')

    for m in range(LO_modes_m):
        for n in range(LO_modes_n):
            slm.update_mode(m,n, zero_shift_angle=False)
            if m==0 and n==0:
                time.sleep(0.5)
            else:
                # in case of higher order modes, transfer previously acquired trace 
                # and do not wait any longer, since this operation takes about 1s for 55k points, 4CH
                scope.transfer_trace(decode=False, save=True, file_path=save_path_oscilloscope_trace)

            # acquire trace for mode (m,n), then set filepath where this trace will be saved (during the next loop)
            scope.acquire_trace()
            save_path_oscilloscope_trace = save_folder + str(seq_n) + '_' + str(m) + '_' + str(n) +'.bin'

    # transfer traces of last mode (m,n)        
    scope.transfer_trace(decode=False, save=True, file_path=save_path_oscilloscope_trace)

    
program_DMD_GUI.park_DMD()

scope.close()
slm.close()


 sequence 22/50

 sequence 23/50

 sequence 24/50

 sequence 25/50

 sequence 26/50

 sequence 27/50

 sequence 28/50

 sequence 29/50

 sequence 30/50
Alignment finished. Min_H, Min_V:  0.0039429570719562435 ,  0.01056950013622604

 sequence 31/50

 sequence 32/50

 sequence 33/50

 sequence 34/50

 sequence 35/50

 sequence 36/50

 sequence 37/50

 sequence 38/50

 sequence 39/50

 sequence 40/50
Alignment finished. Min_H, Min_V:  0.006574658786145937 ,  0.0027508119480185167

 sequence 41/50

 sequence 42/50

 sequence 43/50

 sequence 44/50

 sequence 45/50

 sequence 46/50

 sequence 47/50

 sequence 48/50

 sequence 49/50

 sequence 50/50
Alignment finished. Min_H, Min_V:  0.011956899879927985 ,  0.012825379647196655

 sequence 51/50

 sequence 52/50

 sequence 53/50

 sequence 54/50

 sequence 55/50

 sequence 56/50

 sequence 57/50

 sequence 58/50

 sequence 59/50

 sequence 60/50
Alignment finished. Min_H, Min_V:  0.006564069166686837 ,  0.0022506250335618816

 sequence 61/5

In [ ]:
## estimated time:
n_seq = 28
delta_t = 2*60 + 20

time_per_seq = delta_t / n_seq

total_time = time_per_seq * 76
print('time in for 30k samples: h {:.2}'.format(total_time/60))

In [ ]:
## estimated files size in GB:
size_one_trace_4_ch = 430 #(KB)
expected_binary_file_total_size = size_one_trace_4_ch*256*26*3/(1024*1024)
print('Expected acquired data size: {:.3} GB '.format(expected_binary_file_total_size))